# Imputación Missing Value con Regresión 
##### Mayor información: https://github.com/fmansillaib/python_MTC
##### Personal Page: https://francomansilla.com

#### Paso 1: Cargar la Base de Datos 

In [246]:
import pandas as pd
df = pd.read_excel("/Volumes/GoogleDrive-111868847232940162537/Mi unidad/INVESTIGACION-TRABAJOS/Codigos Stata/mtc/MTC Compartido/Ej. Imputación Regresión.xlsx")
df.head()

,ventas,ing_x1,ing_x2,ing_x3,cost_x1,cost_x2,cost_x3,cost_flag,muestra
0,9107,90.0,1.0,245.70,NaN,0.0,4.30920,1,Train
1,3501,56.0,3.0,NaN,35.28,0.0,2.68128,1,Train
2,4868,NaN,2.0,92.82,21.42,NaN,1.62792,1,Train
3,5436,50.0,NaN,136.50,31.50,NaN,2.39400,1,Train
4,5921,91.0,NaN,248.43,57.33,1.0,4.35708,1,Train


##### ---- ~ -----
##### Nota 1: La variable ventas es la variable target, para una posterior modelelo supervisado.
##### Nota 2: Desde la variable gast_x# hasta cost_x# son variables que tiene missing value (NaN) lo cual habra que imputar.
##### ---- ~ -----

#### Paso 2: Análisis de Missing Value

In [247]:
for i in df.columns.tolist():
    porc_i = round(df[i].isnull().sum()/len(df[i]),4)*100
    print(i+"=", porc_i)

ventas= 0.0
ing_x1= 7.7299999999999995
ing_x2= 6.76
ing_x3= 7.7299999999999995
cost_x1= 11.110000000000001
cost_x2= 9.66
cost_x3= 8.7
cost_flag= 0.0
muestra= 0.0


##### ---- ~ -----
##### Nota 3: Si encontramos una variable sobre un estandar, por ejemplo, 15% de missing value, se puede evaluar eliminar la variable.
##### ---- ~ -----

##### Paso 3: Imputación de Missing Value

##### Paso 3.1. Criterios a considerar:
###### 1. Considera variables con 2 o más valores unicos.
###### 2. Considera variables con menos del umbral de missing value; este caso 15%.
###### 3. Considera variables dicotomicas, es decir, contenga 2 valores discretos (0 ó 1) u otra combinación.
###### 4. Considera variables categoricas, es decir, variable contenga 3 o más valores discretos. 
###### 5. Considera variables categoricas con caracter continuo, es decir, variable contenga 3 o más valores discretos. 

##### ---- ~ -----

##### Paso 3.2. Descripción Imputación
###### La variable ing_x2 tiene 6.72% de missing value. Lo usual en un modelo es: $ventas = \beta_0 + \beta_1*ing _x2$
###### Nota 4: Para realizar la imputación, se utilizara solo la $\textbf{información}$ disponible de $\textbf{ing_x2}$ pero solo de la muestra $\textbf{train}$. 
###### Esto se realizara para realizar el siguiente modelo: $ing_x2 = \beta_0 + \beta_1*ventas$

##### ---- ~ -----

##### Paso 3.3. Código de Imputación

In [248]:
# df a imputar
df_new = df.copy()

# df con valores sin na y filtro
df_var = df.filter(regex = 'ing|cost' , axis = 1)
# o


In [249]:
from sklearn.linear_model import LinearRegression

for i in df_var.columns.tolist():
    
    umbral_unique = 10
    umbral_nan = 0.1
    
    #Criterio 1: Umbral de Missing Value
    porc_i = round(df[i].isnull().sum()/len(df[i]),4)
    
    #Criterio 2: Cantidad de Valores Unicos
    unique_val = df[i].loc[df['muestra']=='Train'].nunique()
    
    # Imputación por Regresión variables continuas y categorias de caracter continuas.
    if porc_i <= umbral_nan and unique_val >= umbral_unique:
        
        print('La variable '+i ,'fue imputada con REGRESIÓN LINEAL', 'con',+unique_val,'valores únicos y',porc_i, 'valores perdidos.' )        
        
        # Regresión Lineal 
        y = df['ventas'].loc[(df[i].notnull()) & (df['muestra']=='Train')].values.reshape(-1, 1)
        X = df[i].loc[(df[i].notnull()) & (df['muestra']=='Train')].values.reshape(-1, 1)

        lr = LinearRegression()
        lr.fit(y, X)
        b0 = lr.intercept_
        b1 = lr.coef_
        
        # Calculo de Mediana en Ventas cuando X(i) es Null
        p50 =  df['ventas'].loc[(df[i].isnull()) & (df['muestra']=='Train')].median()
        
        # Valor de X(i) que deberia ir en el NaN de la variable X(i)
        value_p50 = b0 + b1 * p50
        
        # Reemplazo en el dataframe copy los NaN de la variable X(i)
        df_new[i].fillna(value_p50[0,0], inplace = True)
    
    # Elimina variables con MISSING VALUE mayor al Umbral y un único valor en los valores únicos.
    elif porc_i > umbral_nan or unique_val <= 1:
        
        print('La variable '+i ,'fue ELIMINADA por tener',porc_i, 'valores perdidos.' )        
        df_new.drop(i, axis = 1, inplace = True)
        
    # Imputación por Valor Faltantes a varaibles dicotomicas y variables con menos de 15 categorias
    elif unique_val > 1  & unique_val < umbral_unique:
        print('La variable '+i ,'fue imputada por VALOR FALTANTE', 'con',+unique_val,'valores únicos y',porc_i, 'valores perdidos.' )        
        
        #Calculo máximo valor
        value_max = df[i].max()
        #Valor máximo +1
        value_mas1 = value_max + 1
        #Reemplazar NaN por el Valor asignado.
        df_new[i].fillna(value_mas1, inplace = True)
#FIN

La variable ing_x1 fue imputada con REGRESIÓN LINEAL con 64 valores únicos y 0.0773 valores perdidos.
La variable ing_x2 fue imputada por VALOR FALTANTE con 3 valores únicos y 0.0676 valores perdidos.
La variable ing_x3 fue imputada con REGRESIÓN LINEAL con 67 valores únicos y 0.0773 valores perdidos.
La variable cost_x1 fue ELIMINADA por tener 0.1111 valores perdidos.
La variable cost_x2 fue imputada por VALOR FALTANTE con 2 valores únicos y 0.0966 valores perdidos.
La variable cost_x3 fue imputada con REGRESIÓN LINEAL con 66 valores únicos y 0.087 valores perdidos.
La variable cost_flag fue ELIMINADA por tener 0.0 valores perdidos.


In [250]:
df_new.isnull().sum()

ventas     0
ing_x1     0
ing_x2     0
ing_x3     0
cost_x2    0
cost_x3    0
muestra    0
dtype: int64

In [253]:
df_new.head()

,ventas,ing_x1,ing_x2,ing_x3,cost_x2,cost_x3,muestra
0,9107,90.000000,1.0,245.700000,0.0,4.30920,Train
1,3501,56.000000,3.0,147.960204,0.0,2.68128,Train
2,4868,55.496943,2.0,92.820000,2.0,1.62792,Train
3,5436,50.000000,4.0,136.500000,2.0,2.39400,Train
4,5921,91.000000,4.0,248.430000,1.0,4.35708,Train
